In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars import selectors as cs
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
# from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from typing_extensions import Optional
from data_display.input_data_plots import plot_basin_height_volume_table

from data_display.baseline_plots import *
from utility.pyomo_preprocessing import extract_optimization_results, linear_interpolation_for_bound, arange_float, linear_interpolation_using_cols 
from data_federation.input_model import SmallflexInputSchema
from utility.pyomo_preprocessing import *
from config import settings
from utility.general_function import pl_to_dict, pl_to_dict_with_tuple, build_non_existing_dirs, generate_log
from pyomo_models.input_data_preprocessing import (
    generate_baseline_index, generate_clean_timeseries, generate_water_flow_factor, generate_basin_volume_table,
    clean_hydro_power_performance_table, generate_hydro_power_state, split_timestamps_per_sim, generate_second_stage_state
)
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objects import Figure

from plotly.subplots import make_subplots

import networkx as nx
from pyomo_models.baseline.baseline_input import BaseLineInput
from pyomo_models.baseline.first_stage.first_stage_pipeline import BaselineFirstStage
from pyomo_models.baseline.second_stage.second_stage_pipeline import BaselineSecondStage
COLORS = px.colors.qualitative.Plotly

log = generate_log(name="test")

In [2]:
os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = "/home/ltomasini/gruobi_license/gurobi.lic"


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))

sim_setting: dict = {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True, "with_penalty": True}
baseline_input = BaseLineInput(
    input_schema_file_name=output_file_names["duckdb_input"],
    real_timestep=timedelta(hours=1),
    year=2023,
    hydro_power_mask = c("name").is_in(["Aegina hydro"]),
    volume_factor=1e-6,
)
baseline_first_stage = BaselineFirstStage(
    baseline_input, timestep=timedelta(days=1), turbine_factor=0.7)
baseline_first_stage.solve_model() 

sim_setting = {"quantile": 0, "buffer": 0.2, "powered_volume_enabled": True, "with_penalty": True}
baseline_second_stage = BaselineSecondStage(
    input_instance=baseline_input, 
    first_stage=baseline_first_stage, 
    timestep=timedelta(days=4), 
    log_solver_info=False,
    time_limit=20,
    **sim_setting
    )
# baseline_second_stage.solve_model()


Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.55it/s]
Solving first stage optimization problem: 100%|██████████| 1/1 [00:19<00:00, 19.42s/it]


In [5]:
baseline_second_stage .__dict__

{'real_timestep': datetime.timedelta(seconds=3600),
 'min_datetime': datetime.datetime(2023, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'max_datetime': datetime.datetime(2024, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'year': 2023,
 'market_country': 'CH',
 'market': 'DA',
 'hydro_power_mask': <Expr ['col("name").is_in([Series])'] at 0x7F441368E270>,
 'max_alpha_error': 1.3,
 'volume_factor': 1e-06,
 'solver': <pyomo.solvers.plugins.solvers.GUROBI.GUROBISHELL at 0x7f440c451340>,
 'discharge_flow_measurement': shape: (157_776, 5)
 ┌─────────────────────────┬──────────┬──────────┬──────────────────┬─────┐
 │ timestamp               ┆ river    ┆ value    ┆ discharge_volume ┆ B   │
 │ ---                     ┆ ---      ┆ ---      ┆ ---              ┆ --- │
 │ datetime[μs, UTC]       ┆ str      ┆ f64      ┆ f64              ┆ i32 │
 ╞═════════════════════════╪══════════╪══════════╪══════════════════╪═════╡
 │ 2007-01-01 00:00:00 UTC ┆ Griessee ┆ 0.117833 ┆ 0.000424         ┆ 0   │
 │ 200

In [4]:
fig = plot_second_stage_result(
        simulation_results=baseline_second_stage.simulation_results, time_divider=7*24
    )

fig.show()

AttributeError: 'BaselineSecondStage' object has no attribute 'simulation_results'

In [ ]:
round(baseline_second_stage.simulation_results["income"].sum()/1e6, 3)

3.326